In [ ]:
import zarr
import xarray as xr
import time
import dask

In [ ]:
queue='qt72h'
nb_worker_per_job=40
memory='184GB'
nb_job=2

In [ ]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
import dask.dataframe as dd

cluster = PBSCluster(processes=nb_worker_per_job,cores=nb_worker_per_job, memory=memory,queue=queue,env_extra=['--exclusive'],project='PerfTestPangeo', walltime='00:30:00')
cluster.scale(nb_job*nb_worker_per_job)

c = Client(cluster)

c


In [ ]:
from dask.utils import ensure_dict, format_bytes
    
wk = c.scheduler_info()["workers"]

text="Workers= " + str(len(wk))
memory = [w["memory_limit"] for w in wk.values()]
cores = sum(w["nthreads"] for w in wk.values())
text += ", Cores=" + str(cores)
if all(memory):
    text += ", Memory=" + format_bytes(sum(memory))
print(text)


In [ ]:
%time ds=xr.open_zarr('/work/ALT/odatis/eNATL60/zarr/eNATL60-BLBT02-SSH-1h')

In [ ]:
%time mean=ds.sossheig.mean(dim='time_counter')

In [ ]:
%time mean.load()

In [ ]:
c.close()
cluster.close()